In [1]:
#Set working directory

import os

# Set the working directory
os.chdir(r"G:\My Drive\N  Drive\Projects\Enrollment Campaign\202540")

# Verify current working directory
print(os.getcwd())

G:\My Drive\N  Drive\Projects\Enrollment Campaign\202540


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score

In [3]:
df = pd.read_csv("2024.csv")

df.head()

,ID,PERSON_UID,CUM_GPA,SPRING_GPA,ACADEMIC_STANDING,CLASS,COMPLETION_RATIO,FINANCIAL_HOLD,FULL_TIME,DFW,PELL,ACCOUNT_BALANCE,FINAID_APPLICANT,FGEN,RET
0,797911,787983,1.283,1.000000,1.0,3.0,0.609,1,0,1,0,0.00,0,1,0
1,749298,739359,3.348,4.000000,0.0,4.0,0.831,0,0,0,0,3609.00,0,1,0
2,959819,949922,1.599,1.433333,1.0,2.0,0.641,1,1,1,1,0.00,1,1,0
3,983279,973385,3.425,3.900000,0.0,3.0,1.000,0,1,0,0,5731.71,1,1,0
4,1057344,1047459,3.200,4.000000,0.0,3.0,0.848,1,1,0,0,8932.19,1,1,0


In [4]:
# Create a dictionary with column names as keys and empty strings as values
col_definitions = {col: "" for col in df.columns}
    
print(col_definitions)

{'ID': '', 'PERSON_UID': '', 'CUM_GPA': '', 'SPRING_GPA': '', 'ACADEMIC_STANDING': '', 'CLASS': '', 'COMPLETION_RATIO': '', 'FINANCIAL_HOLD': '', 'FULL_TIME': '', 'DFW': '', 'PELL': '', 'ACCOUNT_BALANCE': '', 'FINAID_APPLICANT': '', 'FGEN': '', 'RET': ''}


In [5]:
# Features (drop identifiers and target)
X = df.drop(columns=["ID", "PERSON_UID", "RET"])

# Target
y = df["RET"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [7]:
X = pd.get_dummies(X, drop_first=True)  # one-hot encode categorical columns

In [8]:
rf = RandomForestClassifier(
    n_estimators=300,   # number of trees
    max_depth=20,     # let trees expand fully
    random_state=42,
    class_weight="balanced"  # helps if retention classes are imbalanced
)

rf.fit(X_train, y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,20
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9086859688195991

Confusion Matrix:
 [[  7  73]
 [  9 809]]

Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.09      0.15        80
           1       0.92      0.99      0.95       818

    accuracy                           0.91       898
   macro avg       0.68      0.54      0.55       898
weighted avg       0.87      0.91      0.88       898



In [10]:
# ----- Prepare features/target -----
X = df.drop(columns=["ID", "PERSON_UID", "RET"])
y = df["RET"]

# One-hot encode BEFORE splitting so columns line up
X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ----- Train RF (same as yours) -----
rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=20,
    random_state=42,
    class_weight="balanced"
)
rf.fit(X_train, y_train)

# ===== Threshold tuning =====
# 1) Get predicted probabilities for RET=1
y_proba = rf.predict_proba(X_test)[:, 1]

# 2) Sweep thresholds, keep two "best" options:
#    (A) closest to expected 15% non-retained
#    (B) best F1 for the minority class (RET=0)
target_not_retained = 0.15
candidates = np.linspace(0.05, 0.95, 37)

best_prevalence_t, best_prevalence_gap = 0.5, 1.0
best_f1_t, best_f1 = 0.5, -1.0

for t in candidates:
    y_pred_t = (y_proba >= t).astype(int)
    pct_not_retained = 1 - y_pred_t.mean()  # share predicted RET=0
    gap = abs(pct_not_retained - target_not_retained)
    if gap < best_prevalence_gap:
        best_prevalence_gap = gap
        best_prevalence_t = t

    # F1 for the negative class (RET=0)
    f1_neg = f1_score(y_test, y_pred_t, pos_label=0)
    if f1_neg > best_f1:
        best_f1 = f1_neg
        best_f1_t = t

print(f"(A) Threshold closest to {target_not_retained:.0%} not-retained: {best_prevalence_t:.2f}")
print(f"(B) Threshold maximizing F1 for RET=0: {best_f1_t:.2f}")

# 3) Choose the prevalence-aligned threshold (or pick best_f1_t if you prefer)
threshold = best_prevalence_t

# 4) Final predictions at chosen threshold + evaluation
y_pred = (y_proba >= threshold).astype(int)

print(f"\nUsing threshold = {threshold:.2f}")
print(f"Predicted % not retained: {(1 - y_pred.mean()):.2%}")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n",
      classification_report(y_test, y_pred, target_names=["Not Retained (0)", "Retained (1)"]))

(A) Threshold closest to 15% not-retained: 0.82
(B) Threshold maximizing F1 for RET=0: 0.82

Using threshold = 0.82
Predicted % not retained: 15.70%
Accuracy: 0.8407572383073497
AUC: 0.7465464547677262

Confusion Matrix:
 [[ 39  41]
 [102 716]]

Classification Report:
                   precision    recall  f1-score   support

Not Retained (0)       0.28      0.49      0.35        80
    Retained (1)       0.95      0.88      0.91       818

        accuracy                           0.84       898
       macro avg       0.61      0.68      0.63       898
    weighted avg       0.89      0.84      0.86       898



In [11]:
df1 = pd.read_csv("2025.csv")

df1.head()

,ID,PERSON_UID,CUM_GPA,SPRING_GPA,ACADEMIC_STANDING,CLASS,COMPLETION_RATIO,FINANCIAL_HOLD,FULL_TIME,DFW,PELL,ACCOUNT_BALANCE,FINAID_APPLICANT,FGEN
0,956190,946292,3.929,4.000000,0.0,3.0,1.000,1,0,0,0,0.00,0,1
1,1027457,1017570,2.917,2.916667,0.0,1.0,0.833,1,1,1,0,9113.27,1,1
2,964734,954837,2.219,1.600000,0.0,2.0,0.578,1,0,1,0,1761.79,0,1
3,1026509,1016622,3.838,0.000000,0.0,1.0,0.517,1,0,1,1,0.00,1,1
4,941976,932078,0.771,1.850000,2.0,2.0,0.172,1,1,1,0,7834.28,0,1


In [12]:
# Drop identifiers and RET (not available in df1)
X_new = df1.drop(columns=["ID", "PERSON_UID"], errors="ignore")

# Apply one-hot encoding
X_new = pd.get_dummies(X_new, drop_first=True)

# Align columns with training data
X_new = X_new.reindex(columns=X_train.columns, fill_value=0)

In [13]:
# Probabilities for RET=1
y_proba_new = rf.predict_proba(X_new)[:, 1]

# Apply same threshold you picked earlier
threshold = 0.82  # <-- replace with your chosen threshold
y_pred_new = (y_proba_new >= threshold).astype(int)

# Add predictions back to df1
df1["RET_PRED"] = y_pred_new
df1["RET_PROB"] = y_proba_new  # optional: keep probability

print(df1["RET_PRED"].value_counts(normalize=True))

RET_PRED
1    0.740276
0    0.259724
Name: proportion, dtype: float64


In [14]:
df1.head()

,ID,PERSON_UID,CUM_GPA,SPRING_GPA,ACADEMIC_STANDING,CLASS,COMPLETION_RATIO,FINANCIAL_HOLD,FULL_TIME,DFW,PELL,ACCOUNT_BALANCE,FINAID_APPLICANT,FGEN,RET_PRED,RET_PROB
0,956190,946292,3.929,4.000000,0.0,3.0,1.000,1,0,0,0,0.00,0,1,0,0.720000
1,1027457,1017570,2.917,2.916667,0.0,1.0,0.833,1,1,1,0,9113.27,1,1,0,0.387293
2,964734,954837,2.219,1.600000,0.0,2.0,0.578,1,0,1,0,1761.79,0,1,0,0.293333
3,1026509,1016622,3.838,0.000000,0.0,1.0,0.517,1,0,1,1,0.00,1,1,0,0.733333
4,941976,932078,0.771,1.850000,2.0,2.0,0.172,1,1,1,0,7834.28,0,1,0,0.646667


In [15]:
df2 = pd.read_csv("appointment_summaries_report.csv")

df2.head()

,Student ID,Summary
0,1021203.0,Delaney in for Summer advising and Registratio...
1,1021203.0,Delaney in for Fall advising and Registration....
2,1001456.0,"Gabe wanted to discuss a 25-26 academic plan, ..."
3,1001456.0,Gabe was hit by a car last quarter and had a c...
4,1001456.0,Gabe is considering changing his major to MENG...


In [16]:
# Group by Student ID and concatenate all summaries into one string
df2 = (
    df2.groupby("Student ID", as_index=False)
       .agg({"Summary": lambda x: " ".join(x.astype(str))})
)

# Preview
df2.head()

,Student ID,Summary
0,4665.0,Student met with advisor to discuss fall quart...
1,59091.0,Met with Missy for a scheduled appt. Missy wan...
2,60051.0,Met with student to talk about returning to EW...
3,77697.0,Student met with advisor to discuss switching ...
4,83574.0,Brandy is returning to complete ITDS degree. S...


In [17]:
# Merge df1 (predictions, etc.) with df2 (summaries per student)
merged_df = pd.merge(
    df1,
    df2,
    how="left",                # keep all students from df1
    left_on="ID",              # column in df1
    right_on="Student ID"      # column in df2
)

# Optional: drop the duplicate "Student ID" if you only want one identifier
merged_df = merged_df.drop(columns=["Student ID"])

merged_df.head()

,ID,PERSON_UID,CUM_GPA,SPRING_GPA,ACADEMIC_STANDING,CLASS,COMPLETION_RATIO,FINANCIAL_HOLD,FULL_TIME,DFW,PELL,ACCOUNT_BALANCE,FINAID_APPLICANT,FGEN,RET_PRED,RET_PROB,Summary
0,956190,946292,3.929,4.000000,0.0,3.0,1.000,1,0,0,0,0.00,0,1,0,0.720000,NaN
1,1027457,1017570,2.917,2.916667,0.0,1.0,0.833,1,1,1,0,9113.27,1,1,0,0.387293,Marisol wanted to register for Summer and Fall...
2,964734,954837,2.219,1.600000,0.0,2.0,0.578,1,0,1,0,1761.79,0,1,0,0.293333,NaN
3,1026509,1016622,3.838,0.000000,0.0,1.0,0.517,1,0,1,1,0.00,1,1,0,0.733333,NaN
4,941976,932078,0.771,1.850000,2.0,2.0,0.172,1,1,1,0,7834.28,0,1,0,0.646667,NaN


In [18]:
merged_df.to_csv("registration_status.csv", index=False)